In [ ]:
import pandas_ta as ta
import yfinance as yf
import pandas as pd
import pandas_datareader as pdr
from datetime import datetime, timedelta

In [ ]:
def get_ema(name):
    # Calculate the date range for the months
    end_date = datetime.today()
    start_date = end_date - timedelta(days=1000)
    df = yf.download(name, start=start_date, end=end_date)[['Close', 'Low']]
    
    if 'Dividends' in df.columns:
        df = df.drop(columns=['Dividends'])
    

    # Get the 12-day EMA of the closing price
    ema_12 = df['Close'].ewm(span=12, adjust=False, min_periods=12).mean()

    # Get the 144-day EMA of the closing price
    ema_144 = df['Close'].ewm(span=144, adjust=False, min_periods=144).mean()

    # Get the 169-day EMA of the closing price
    ema_169 = df['Close'].ewm(span=169, adjust=False, min_periods=169).mean()
    
    # Get the 576-day EMA of the closing price
    ema_576 = df['Close'].ewm(span=576, adjust=False, min_periods=576).mean()
    
    # Get the 676-day EMA of the closing price
    ema_676 = df['Close'].ewm(span=676, adjust=False, min_periods=676).mean()


    # Add all of our new values for the MACD to the dataframe
    df['ema_12'] = df.index.map(ema_12)
    df['ema_144'] = df.index.map(ema_144)
    df['ema_169']= df.index.map(ema_169)
    df['ema_576'] = df.index.map(ema_576)
    df['ema_676'] = df.index.map(ema_676)
    
    # View our data
    pd.set_option("display.max_columns", None)
    
    return df

In [ ]:
def trigger(df):
    trigger_flag=False
    df_today=df.iloc[-1]
    five_day_low=df.iloc[-5:]["Low"]  
    breakout=True
    
    ema_12=df_today["ema_12"]
    ema_144=df_today["ema_144"]
    ema_169=df_today["ema_169"]
    ema_576=df_today["ema_576"]
    ema_676=df_today["ema_676"]
    close_p = df_today['Close']
    low_p= df_today['Low']
    
    if(ema_144 > ema_576 and ema_12>ema_144):
    
        for i in range(0,len(five_day_low)):

            if(five_day_low[i]<ema_144):
                breakout=False
                break
        
            
        if(breakout==True and low_p < ema_144):
                trigger_flag=True

        
    return trigger_flag

In [ ]:
def send_slack_message(webhook_url, message):
    payload = {"text": message}
    headers = {"Content-Type": "application/json"}
    
    response = requests.post(webhook_url, json=payload, headers=headers)
    
    if response.status_code == 200:
        print("Message sent successfully")
    else:
        print(f"Failed to send message: {response.status_code}, {response.text}")

In [ ]:
compName=pd.read_csv("constituents.csv")


coName=compName["Symbol"]
coType=compName["GICS Sector"]
message=[]

coName_list=coName.tolist()
coType_list=coType.tolist()

i=0
for i in range(len(coName.index)):
    df=get_ema(coName_list[i])
    if(df.empty):
        print("\nSymbol= "+coName_list[i]+" data not found")
    
    else:
        flag=trigger(df)
        if(flag==True):
            print("\nSymbol= "+coName_list[i])
            print("\nSector= "+coType_list[i])
            message = message + " " + coName_list[i]
            display(df)

# Replace with your webhook URL
webhook_url = "YOUR OWN URL"


send_slack_message(webhook_url, message)
